# <font color='red'>Backtest vectorizado para una estrategia 'Regresion lineal' definida.</font>

### Hackeado by J3viton (learning BackTesting 2021).

---

En este notebook vamos a automatizar el backtesting para estrategias. Previamente tenemos que definir **la estrategia** usando el template (import estrategia_XX as rg), donde definimos la clase estrategia que contiene los metodos de analisis, entreda y salida de mercado. Este módulo se programa y adapta para cada estrategia. Luego en el metodo 'analisis' se crea un fichero excel con las metricas y rendimientos para evaluar la estrategia.

Vamos a usar el módulo Pandas, para realizar un backtest vectorizado, es decir calcular el resultado del backtest sin iterar sobre las filas (barras) de la serie temporal.

Posteriormente graficamos y mostramos los parametros generales que nos permiten analizar la estrategia.

#### J3Viton  2021

link a la base:

https://github.com/Python-para-Trading/Webinars-Docs/blob/master/Webinar%202/Webinar%202%20-%20Backtest%20con%20Pandas%20v%202.ipynb.

---

# DEFINICION DEL SISTEMA

Como es usual, importamos los módulos que vamos a necesitar para las distintas tareas, y preparamos Matplotlib para una mejor visualización de los gráficos en el notebook.

Se importa un modulo llamado analisis, en realidad se trata de un archivo .py con algunas funciones de apoyo para el trabajo, debe estar guardado en la misma carpeta que este notebook.
Se importa el módulo 'estrategia_XX', donde se define la estrategia de entreda y salida

Es importante tener clara la lógica del sistema, pues debemos plasmarla de forma sencilla y correcta en el dataframe para realizar el backtest del mismo.


In [3]:
import analisis
import regresionAMedia as rg  #cambiar segun el módulo con la estrategia implementado
import pandas as pd
import datetime as dt
from time import time
import yfinance as yf
import numpy as np

{'id': 1473252352, 'first_name': 'vital_bot', 'is_bot': True, 'username': 'vital_quant_bot', 'can_join_groups': True, 'can_read_all_group_messages': False, 'supports_inline_queries': False}


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 18, 8
plt.style.use('ggplot')

from IPython.display import display, Markdown

***0.- RECOGIDA DE DATOS INICIALES***
Introducimos el instrumento y las fechas


In [5]:
instrumento_ = 'k'   # ticker del valor en yahooFinance

Fecha_Cominezo_Backtesting = dt.datetime(2015,1,2)
Fecha_Final_Backtesting    = dt.datetime(2022,9,15)
Fecha_Final_Backtesting    = dt.datetime.today()




***1.- Rango fechas a analizar***

Definimos el rango global de datos historicos que vamos a evaluar.
Definimos la ventana que vamos a ir desplazando por todo el espectro para ir analilazando como se comporta la estrategia. La funion 'analisis' nos vale para tiempo real y para backtesting, para back le pasamos la ventana como si la fecha de fin de la ventana fuera la fecha de hoy.
Tener en cuenta que la 'ventana' tiene uqe tener una anchura que nos permita hacer los calculos en rolling (ejemplo EMA 200)

In [6]:
# Rango completo para backTesting
#start2 =dt.datetime(2008,1,2)
start2= Fecha_Cominezo_Backtesting 
#end2   =dt.datetime(2021,11,18)
end2= Fecha_Final_Backtesting 
start_G= start2.strftime("%Y-%m-%d")
end_G  =   end2.strftime("%Y-%m-%d")
TOTAL_len= (end2-start2).days
print('Tamaño timeseries global a analizar:  ', TOTAL_len, 'sesiones')

#ventana de analisis 200 sesiones
startWindow2 = start2  #dt.datetime(2008,1,5)
endWindow2   =startWindow2 + dt.timedelta(days=500) #ventana grande para que se puedan hacer los calculos de EMA200
startWindow= startWindow2.strftime("%Y-%m-%d")
endWindow  =   endWindow2.strftime("%Y-%m-%d")
window_len= (endWindow2-startWindow2).days
print('Tamaño de la ventana a analizar paso a paso:  ', window_len, 'sesiones')

Tamaño timeseries global a analizar:   2836 sesiones
Tamaño de la ventana a analizar paso a paso:   500 sesiones


In [7]:
 #dff = pd.DataFrame(columns=('Close','Volume', 'Senal', 'Dif_Close', 'Retornos','Capital'))

***2.- Descargamos los datos para el marco Global***

In [8]:
instrumento = instrumento_  # 'rovi.mc'  #Vamos título a título. Mejora: Conjunto de títulos

In [9]:
dff = yf.download(instrumento, start_G,end_G)

[*********************100%***********************]  1 of 1 completed


In [10]:
dff.dropna(inplace=True)  
dff.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,65.629997,66.139999,65.089996,65.480003,50.660633,846800
2015-01-05,65.239998,65.370003,64.139999,64.260002,49.716743,1360200
2015-01-06,64.529999,65.089996,64.209999,64.449997,49.863747,2198100
2015-01-07,65.099998,66.089996,65.070000,66.010002,51.070690,1683400
2015-01-08,66.480003,67.250000,66.370003,67.209999,51.999111,1422800


***3.- Creamos la clase strategy que tiene toda la lógica***

In [11]:
regreMedia= rg.StrategyClass(back=True)    #Creamos la clase

***4.- Recorremos el dataframe con valores buscando las señales de la estrategia***

In [12]:
dff.index

DatetimeIndex(['2015-01-02', '2015-01-05', '2015-01-06', '2015-01-07',
               '2015-01-08', '2015-01-09', '2015-01-12', '2015-01-13',
               '2015-01-14', '2015-01-15',
               ...
               '2022-09-26', '2022-09-27', '2022-09-28', '2022-09-29',
               '2022-09-30', '2022-10-03', '2022-10-04', '2022-10-05',
               '2022-10-06', '2022-10-07'],
              dtype='datetime64[ns]', name='Date', length=1956, freq=None)

## Proceso de backTesting ##

En este 'for' vamos recorriendo la muestra del historico de datos (TOTAL), desplazando una sesión hacia el futuro 
en cada iteracion. Vamos pasando la movilola del pasado sesión a sesión por el análisis descrito en Strategy class.

Ouput:
El sistema registra los siguientes parámetros:

.-
.-
.-


In [13]:
startWindow

'2015-01-02'

In [14]:
tiempo_inicial = time()   # Tomamos tiempos para ver cuanto tarda en hacer la estrategia

In [15]:
dfe = pd.DataFrame({'A' : []})   #df empty

***En este 'for' desplazamos la ventana sesión a sesión a lo largo de todo el rango de fechas. Dejamos que las funciones de la clase estrategia hagan el trabajo de comprar//vender y anotar***


In [16]:
#Para pruebas
#TOTAL_len =1000

for i in range(TOTAL_len):
    endWindow3   =endWindow2 + dt.timedelta(days=i) 
    endWindow    =endWindow3.strftime("%Y-%m-%d")
    print ('end date:', endWindow)
    
    if(endWindow in dff.index):
        df_aux= dff.loc[startWindow:endWindow]       #voy pasando los datos desplazando la ventana
        
        recogo = regreMedia.analisis(instrumento, startWindow, endWindow, df_aux) #Llamada a la clase estrategia. LA CLAVE DE TODO!!!
        
        print ('................................................Analizando, muestra', i, 'de', TOTAL_len, 'fecha', endWindow)
        
            
    else:
        print('..............Día sin sesión, next please')

        

end date: 2016-05-16
aqui estoy en out
 datos desde archivo
................................................Analizando, muestra 0 de 2836 fecha 2016-05-16
end date: 2016-05-17
 Datos desde archivo
Coefficients: 
 [0.03963804]
Independent term: 
 61.66535730265508
Coefficients: 
 [0.05642023]
Independent term: 
 65.91458676238558
Coefficients: 
 [0.04440827]
Independent term: 
 65.29667263273659
   Instrumento  long_short_out                    date      precio  beneficio  \
0       IBE.MC               1 2011-08-10 00:00:00.000  123.000000        NaN   
1         ABBV               0 2022-07-31 10:50:32.080  143.509995  19.632078   
2         abbv               0 2022-04-29 00:00:00.000  146.880005  11.802326   
3         VRTX               0 2022-08-01 00:00:00.000  273.829987  21.637584   
4          LLY               0 2022-08-09 14:03:44.253  302.500000  19.262965   
5          lly               0 2022-08-05 00:00:00.000  301.320007  20.341828   
6         vrtx               0 2022

 Datos desde archivo
Coefficients: 
 [0.03961245]
Independent term: 
 61.66830261591548
Coefficients: 
 [0.0560547]
Independent term: 
 66.03450767175475
Coefficients: 
 [0.04456204]
Independent term: 
 65.28911370708121
   Instrumento  long_short_out                    date      precio  beneficio  \
0       IBE.MC               1 2011-08-10 00:00:00.000  123.000000        NaN   
1         ABBV               0 2022-07-31 10:50:32.080  143.509995  19.632078   
2         abbv               0 2022-04-29 00:00:00.000  146.880005  11.802326   
3         VRTX               0 2022-08-01 00:00:00.000  273.829987  21.637584   
4          LLY               0 2022-08-09 14:03:44.253  302.500000  19.262965   
5          lly               0 2022-08-05 00:00:00.000  301.320007  20.341828   
6         vrtx               0 2022-08-01 00:00:00.000  273.829987  21.637584   
7          CMS               0 2022-09-04 10:50:36.880   67.139999   2.785874   
8          cms               1 2022-09-02 00:00:00

Coefficients: 
 [0.05600517]
Independent term: 
 66.09538512728108
Coefficients: 
 [0.04468242]
Independent term: 
 65.28311624077723
   Instrumento  long_short_out                    date      precio  beneficio  \
0       IBE.MC               1 2011-08-10 00:00:00.000  123.000000        NaN   
1         ABBV               0 2022-07-31 10:50:32.080  143.509995  19.632078   
2         abbv               0 2022-04-29 00:00:00.000  146.880005  11.802326   
3         VRTX               0 2022-08-01 00:00:00.000  273.829987  21.637584   
4          LLY               0 2022-08-09 14:03:44.253  302.500000  19.262965   
5          lly               0 2022-08-05 00:00:00.000  301.320007  20.341828   
6         vrtx               0 2022-08-01 00:00:00.000  273.829987  21.637584   
7          CMS               0 2022-09-04 10:50:36.880   67.139999   2.785874   
8          cms               1 2022-09-02 00:00:00.000   67.139999   2.785874   
9         TMUS               1 2022-09-14 10:50:03.523  

 Datos desde archivo
Coefficients: 
 [0.03941252]
Independent term: 
 61.691526082632734
Coefficients: 
 [0.05624255]
Independent term: 
 66.15981653014225
Coefficients: 
 [0.04480907]
Independent term: 
 65.27670327089332
   Instrumento  long_short_out                    date      precio  beneficio  \
0       IBE.MC               1 2011-08-10 00:00:00.000  123.000000        NaN   
1         ABBV               0 2022-07-31 10:50:32.080  143.509995  19.632078   
2         abbv               0 2022-04-29 00:00:00.000  146.880005  11.802326   
3         VRTX               0 2022-08-01 00:00:00.000  273.829987  21.637584   
4          LLY               0 2022-08-09 14:03:44.253  302.500000  19.262965   
5          lly               0 2022-08-05 00:00:00.000  301.320007  20.341828   
6         vrtx               0 2022-08-01 00:00:00.000  273.829987  21.637584   
7          CMS               0 2022-09-04 10:50:36.880   67.139999   2.785874   
8          cms               1 2022-09-02 00:00:

   Instrumento  long_short_out                    date      precio  beneficio  \
0       IBE.MC               1 2011-08-10 00:00:00.000  123.000000        NaN   
1         ABBV               0 2022-07-31 10:50:32.080  143.509995  19.632078   
2         abbv               0 2022-04-29 00:00:00.000  146.880005  11.802326   
3         VRTX               0 2022-08-01 00:00:00.000  273.829987  21.637584   
4          LLY               0 2022-08-09 14:03:44.253  302.500000  19.262965   
5          lly               0 2022-08-05 00:00:00.000  301.320007  20.341828   
6         vrtx               0 2022-08-01 00:00:00.000  273.829987  21.637584   
7          CMS               0 2022-09-04 10:50:36.880   67.139999   2.785874   
8          cms               1 2022-09-02 00:00:00.000   67.139999   2.785874   
9         TMUS               1 2022-09-14 10:50:03.523  137.740005   7.525300   
10         NOC               1 2022-09-14 10:50:03.523  468.929993  31.295953   
11         MCK              

...

Empty DataFrame
Columns: [A]
Index: []
................................................Analizando, muestra 1250 de 2836 fecha 2019-10-18
end date: 2019-10-19
..............Día sin sesión, next please
end date: 2019-10-20
..............Día sin sesión, next please
end date: 2019-10-21
 Datos desde archivo
Coefficients: 
 [-0.00902518]
Independent term: 
 73.17136819462138
Coefficients: 
 [0.03379277]
Independent term: 
 55.08391947105749
Coefficients: 
 [-0.02936037]
Independent term: 
 64.04417150512309
Empty DataFrame
Columns: [A]
Index: []
................................................Analizando, muestra 1253 de 2836 fecha 2019-10-21
end date: 2019-10-22
 Datos desde archivo
Coefficients: 
 [-0.00902592]
Independent term: 
 73.17166484477123
Coefficients: 
 [0.03420301]
Independent term: 
 55.065650801587466
Coefficients: 
 [-0.0287279]
Independent term: 
 63.95483299066007
Empty DataFrame
Columns: [A]
Index: []
................................................Analizando, muestra 125

 Datos desde archivo
Coefficients: 
 [-0.00893228]
Independent term: 
 73.13350719824449
Coefficients: 
 [0.0473793]
Independent term: 
 54.21715954111583
Coefficients: 
 [-0.01670353]
Independent term: 
 62.34262200304652
Empty DataFrame
Columns: [A]
Index: []
................................................Analizando, muestra 1281 de 2836 fecha 2019-11-18
end date: 2019-11-19
 Datos desde archivo
Coefficients: 
 [-0.00891974]
Independent term: 
 73.12837459603033
Coefficients: 
 [0.04836427]
Independent term: 
 54.150055080812365
Coefficients: 
 [-0.01605609]
Independent term: 
 62.26110392421832
Empty DataFrame
Columns: [A]
Index: []
................................................Analizando, muestra 1282 de 2836 fecha 2019-11-19
end date: 2019-11-20
 Datos desde archivo
Coefficients: 
 [-0.00890699]
Independent term: 
 73.12315111368444
Coefficients: 
 [0.04935194]
Independent term: 
 54.08288197873244
Coefficients: 
 [-0.01540526]
Independent term: 
 62.17972858147717
Empty DataFr

 Datos desde archivo
Coefficients: 
 [-0.00865942]
Independent term: 
 73.02088217557444
Coefficients: 
 [0.05820457]
Independent term: 
 54.043195476247305
Coefficients: 
 [-0.00392751]
Independent term: 
 60.83714057783474
Empty DataFrame
Columns: [A]
Index: []
................................................Analizando, muestra 1310 de 2836 fecha 2019-12-17
end date: 2019-12-18
 Datos desde archivo
Coefficients: 
 [-0.00864302]
Independent term: 
 73.0140590044566
Coefficients: 
 [0.0583877]
Independent term: 
 54.085023780367266
Coefficients: 
 [-0.00330745]
Independent term: 
 60.76995533606889
Empty DataFrame
Columns: [A]
Index: []
................................................Analizando, muestra 1311 de 2836 fecha 2019-12-18
end date: 2019-12-19
 Datos desde archivo
Coefficients: 
 [-0.00862551]
Independent term: 
 73.00677082057295
Coefficients: 
 [0.05857197]
Independent term: 
 54.1290894830049
Coefficients: 
 [-0.00269174]
Independent term: 
 60.703877068874235
Empty DataFr

 Datos desde archivo
Coefficients: 
 [-0.00816996]
Independent term: 
 72.81562709804444
Coefficients: 
 [0.06756761]
Independent term: 
 54.45427306104062
Coefficients: 
 [0.00808731]
Independent term: 
 59.66423896390652
Empty DataFrame
Columns: [A]
Index: []
................................................Analizando, muestra 1340 de 2836 fecha 2020-01-16
end date: 2020-01-17
 Datos desde archivo
Coefficients: 
 [-0.00813862]
Independent term: 
 72.80237961736627
Coefficients: 
 [0.06833479]
Independent term: 
 54.449038765181356
Coefficients: 
 [0.00868679]
Independent term: 
 59.61234642461572
Empty DataFrame
Columns: [A]
Index: []
................................................Analizando, muestra 1341 de 2836 fecha 2020-01-17
end date: 2020-01-18
..............Día sin sesión, next please
end date: 2020-01-19
..............Día sin sesión, next please
end date: 2020-01-20
..............Día sin sesión, next please
end date: 2020-01-21
 Datos desde archivo
Coefficients: 
 [-0.0081084

KeyError: 'long_short_out'

***Recogemos los datos de las entradas que ha realizado la Estrategia***

In [ ]:
data=rg.StrategyClass.dfLog
#data.dropna(inplace=True) 

In [ ]:
data.tail()

In [ ]:
data['Senal'].plot(title = 'Señales de compra de la estrategia ' )
# Ploteando una parte
#data.loc['2010':'2022','Senal'].plot(title = 'Señales Regresión a la media ' +instrumento,xlim=('2010','2022'))

In [ ]:
tiempo_final = time() 
 
tiempo_ejecucion = (tiempo_final - tiempo_inicial)/60
 
print ('El tiempo de ejecucion fue:',tiempo_ejecucion,"minutos") #En segundos

In [ ]:
data.loc['2011-01-04':'2019-11-04']

In [ ]:
#Guardamos el resultado del analisis en un pickle
#dff.to_pickle("./primerBack_IBE.pkl")

***3.- Ingeniería de datos para calcular la bondad de la estrategia***

In [ ]:

data['Dif_Close'] = data.Price.pct_change()
data['Retornos'] = data.Dif_Close * data.Senal.shift(1)   
data['Capital'] = (data.Retornos + 1).cumprod() * 100
    
rg.StrategyClass.dfLog=data  #Ojo a esta liena, no me cuadra.

#quant_j.salvarExcel(StrategyClass.dfLog, "log"+instrumento)   
#data.to_pickle('almacen')    #df = pd.read_pickle(file_name)

>Para calcular los retornos del sistema, calculamos la diferencia relativativa entre el precio de cierre y el del día anterior. Para posteriormente multiplicarlo por la señal que del día anterior que nos indicaba la posición a tomar (si estaba comprado, sumo beneficio).

In [ ]:
data.tail()

In [ ]:
data[60:]

>El capital, por comodidad, lo calcularemos en base 100, es decir como si iniciaramos la inversión con 100 unidades monetarias. Para su calculo arrastramos el producto acumulado de los retornos mas 1, multiplicados como dijimos por 100.

Pero para observar gráficamente la evolución del sistema necesitamos un gráfico con mas información. Para ello definimos una función que nos mostrará un gráfico con la evolución de nuestra estrategia, comparandola con la del activo. En un subgráfico se visualizará el drawdown del sistema, comparado de nuevo con el del activo. Y por útlimo las posiciones que toma el sistema.

In [ ]:
def grafico (df):
    estudio = df.copy()
    
    DD_bh, maxDD, maxDD_ini, maxDD_fin = analisis.DrawDown(estudio.Dif_Close[60:], info = False) 
    
    DD, maxDD, maxDD_ini, maxDD_fin = analisis.DrawDown(estudio.Retornos.fillna(0), info = False) 

    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(18, 12), gridspec_kw = {'height_ratios':[3, 1, 1]})
    fig.suptitle('Estrategia vs B & H', fontsize=20)

    #Primer plot analiza la evolcuion de una inversion de 100€ en buy&hold y la estrategia)
    ax1.plot(estudio.Capital)
    ax1.plot((estudio.Price) * (100/estudio.Price[1]))
    ax1.set_title('Capital')
    ax1.legend(['Estrategia','Buy & Hold'])

    ax2.plot(DD*100, c='0.5')
    ax2.plot(DD_bh*100, c='y')
    ax2.fill_between(DD.index, 0, DD*100, color='0.7')
    ax2.set_title('Drawdown')
    ax2.legend(['Estrategia','Buy & Hold'])

    ax3.plot(estudio.Senal, c='orange')
    #ax3.fill_between(estudio.index, 0, estudio.Senal*100, color='orange')
    ax3.set_title('Posición')

    plt.show()
    return

In [ ]:
print(instrumento)

In [ ]:
grafico(data)

---

In [ ]:
#data[55:70]

---

# ANALISIS DEL SISTEMA

In [ ]:
Beneficio_Bruto = data.Capital[-1] - data.Capital[1]
print ('Beneficio Bruto : {:.2f} €  con una inverison de 100€'.format(Beneficio_Bruto))

In [ ]:
#Rentabilidad anual compuesta. Calculamos lo que la inversion se ha incrementado cada año con el interes compuesto
CAGR = analisis.CAGR(data.Retornos)
print ('CAGR : {:.2f}%'.format(CAGR * 100))

In [ ]:
#Ratio calculado restando una rentabilidad segura a nuestra rentabilidad de estrategia y dividido por el riego (=volatilidad)
data.Retornos=data.Retornos.replace(0,np.e)  #Quito los cero (no sepuede hacer logariotmo de cero), pero no se que poner
Sharpe = CAGR / (np.log(data.Retornos + 1).std(skipna=True) * 252 ** 0.5)
print ('Sharpe : {:.3f}'.format(CAGR ))

In [ ]:
# Actualizo la excel con los economic
regreMedia.analisisEconomics(instrumento)

---


In [ ]:
# 

### Cálculo esperanza matemática
 
 (%Aciertos * beneficioMedio )-(%Errores * perdidaMedia)
 
 Nos da idea el benefcio esperado por entrada

In [ ]:
print("Numero operaciones con beneficio ->", 
      data[data['ExitReason'] ==1]['ExitReason'].count()) 
print("Numero operaciones con perdidas ->", 
      data[data['ExitReason'] ==-1]['ExitReason'].count()) 
  
data[data['ExitReason']>0]['ExitReason'] 

data[data['Senal']>0][1:10]
data[data['Senal']>0].index[1]

In [ ]:
# ENTRADAS
print (data[(data['Senal']>0) & (data['Beneficio']>0)].index[0])
data[(data['Senal']>0) & (data['Beneficio']>0) ][0:5] 
# SALIDA PERDIDAS
print (data[(data['Senal'].shift(1) >0) & (data['ExitReason']== -1)].index[0])
data[(data['Senal'].shift(1) >0) & (data['ExitReason']== -1)][0:5]
# SALIDA GANANCIAS
data[(data['Senal'].shift(1) >0) & (data['ExitReason']== 1)][0:5]

In [ ]:
serieIndicesENTRADA=data[(data['Senal']>0) & (data['Beneficio']>0)].index

print(serieIndicesENTRADA.size)

serieIndicesENTRADA

serieIndicesEXIT=data[(data['ExitReason']==1) | (data['ExitReason']==-1)].index

print(serieIndicesEXIT.size)

#data['ExitReason'][serieIndicesEXIT[2]]

In [ ]:
serieIndicesENTRADA

In [ ]:
#Cálculo Esperanza Matemática
countGanando=0
countPerdiendo=0

bolsaGanando=0
bolsaPerdiendo=0

for i in range(serieIndicesEXIT.size):
    if(data['ExitReason'][serieIndicesEXIT[i]] == -1):
        ref1  = data['Price'][serieIndicesEXIT[i]]
        ref2 = data['Price'][serieIndicesENTRADA[i]]
        print ("-1",ref1, ref2)
        countPerdiendo= 1+countPerdiendo
        bolsaPerdiendo= bolsaPerdiendo + (ref1-ref2)

    
    if(data['ExitReason'][serieIndicesEXIT[i]] == 1):
        ref1  = data['Price'][serieIndicesEXIT[i]]
        ref2 = data['Price'][serieIndicesENTRADA[i]]
        print ("+1",ref1, ref2)
        countGanando= 1+countGanando
        bolsaGanando= bolsaGanando + (ref1-ref2)
    
#(%Aciertos * beneficioMedio )-(%Errores * perdidaMedia)

count=countGanando+countPerdiendo


print ("bolsa ganando   --> ", bolsaGanando, "   Número operaciones ganando   ", countGanando)   
print ("bolsa perdiendo -->", bolsaPerdiendo, "   Número operaciones perdiendo", countPerdiendo) 
print (" Media perdiendo --> ", (bolsaPerdiendo/countPerdiendo),"\n", "Media ganando --> ", (bolsaGanando/countGanando)) 


esperanza = (countGanando/count *(bolsaGanando/countGanando))+(countPerdiendo/count *(bolsaPerdiendo/countPerdiendo))
print("")
print(instrumento)
print("ESPERANZA MATEMATICA ES: ", esperanza)



---